In [40]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd
import numpy as np

import nltk

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import MinMaxScaler

from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import GridSearchCV

#### Sistema de recomendación item-item:

- **def recomendacion_item_item(busines_id):**

> Ingresando el id de un negocio, deberíamos recibir una lista con 5 restaurantes recomendados similares a dicho negocio.

In [73]:
# Ruta del archivo
file_path = r'./data/merged.csv'

# Leer el archivo en un DataFrame
restaurantes = pd.read_csv(file_path)

In [64]:
restaurantes.columns

Index(['name', 'business_id', 'latitude', 'longitude', 'category',
       'avg_rating', 'num_of_reviews', 'user_id', 'rating', 'timestamp',
       'sentiment_analysis'],
      dtype='object')

In [74]:
# Nos quedamos con las columnas relevantes
restaurantes = restaurantes[['name','category','avg_rating','num_of_reviews']]

Primero vamos a filtrar solo las categorais de negocios relacionadas al rubro gastronomico

In [75]:
# Seleccionamos solo los valores únicos de categoría y los convertimos en cadenas de texto
categorias_unicas = restaurantes['category'].astype(str).unique()

# Convertimos el array de valores únicos en una lista de cadenas de texto
categorias_unicas = categorias_unicas.tolist()

import spacy

# Cargar el modelo en inglés de spaCy
nlp = spacy.load("en_core_web_md")

# Palabras clave a comparar
words = ['restaurant', 'food', 'cuisine']

# Inicializar una lista vacía para almacenar las categorías relacionadas
related_categories = []

# Iterar sobre cada categoría en categorias_unicas
for category in categorias_unicas:
    # Inicializar el puntaje de similaridad máximo
    max_similarity = 0.0
    # Iterar sobre cada palabra en words
    for word in words:
        # Calcular la similitud entre la categoría y la palabra
        similarity = nlp(category).similarity(nlp(word))
        # Actualizar el puntaje de similaridad máximo
        max_similarity = max(max_similarity, similarity)
    # Si el puntaje de similaridad máximo es mayor a un umbral, agregar la categoría a related_categories
    if max_similarity > 0.7:
        related_categories.append(category)


C:\Users\sebas\AppData\Local\Temp\ipykernel_7492\1941504441.py:25: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = nlp(category).similarity(nlp(word))


In [78]:
# Nos quedamos solo con las filas cuya categoría esté en related_categories
restaurantes = restaurantes[restaurantes['category'].isin(related_categories)]

In [79]:
# Observamos el tamaño resultante
restaurantes.shape

(88219, 4)

In [102]:
restaurantes.head(5)

,name,category,avg_rating,num_of_reviews
592,Bachata Rosa,62,3.6,8
593,Bachata Rosa,62,3.6,8
594,Bachata Rosa,62,3.6,8
788,Zampini's Bottega,62,4.7,18
789,Zampini's Bottega,62,4.7,18


In [103]:
from sklearn.neighbors import KNeighborsTransformer

# Crear un nuevo DataFrame con las columnas 'avg_rating' y 'num_of_reviews'
X = restaurantes[['avg_rating', 'num_of_reviews']].copy()

# Definir el modelo KNeighborsTransformer
model = KNeighborsTransformer(n_neighbors=5, metric='euclidean', n_jobs=-1, algorithm='auto', leaf_size=30, p=2, metric_params=None)

# Ajustar el modelo a los datos
model.fit(X)


KNeighborsTransformer(metric='euclidean', n_jobs=-1)

In [113]:
import pickle

# Guardar el modelo en un archivo .pkl
with open('item_item.pkl', 'wb') as file:
    pickle.dump(model, file)


In [116]:
import pickle
from collections import deque

def get_similar_restaurants(restaurant_name, n_restaurants=5):
    # Cargar el modelo desde el archivo .pkl
    with open('./models/modelo_item_item.pkl', 'rb') as file:
        model = pickle.load(file)

    restaurant_index = restaurantes[restaurantes['name'] == restaurant_name].index[0]
    restaurant_features = X.iloc[restaurant_index].values.reshape(1, -1)

    # Encontrar los índices de los restaurantes más cercanos
    distances, indices = model.kneighbors(restaurant_features, n_neighbors=len(X))

    # Eliminar el índice del propio restaurante de la lista de índices similares
    similar_indices = np.delete(indices[0], 0)

    # Crear una cola con los índices similares
    queue = deque(similar_indices)

    # Seleccionar los primeros n_restaurants índices no repetidos
    selected_indices = []
    while len(selected_indices) < n_restaurants:
        index = queue.popleft()
        if index not in selected_indices:
            selected_indices.append(index)
        else:
            queue.append(index)

    # Obtener los nombres de los restaurantes similares
    similar_restaurants = restaurantes.iloc[selected_indices]['name'].tolist()

    return similar_restaurants


In [117]:
get_similar_restaurants("Bachata Rosa")

c:\Users\sebas\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KNeighborsTransformer was fitted with feature names
  warnings.warn(


['La Granja',
 'Tienda El Castillo',
 'Villa Italian Kitchen',
 'Krystal - Closed',
 'Sushigami']

In [118]:
# Exportamos la data para el modelo de ML
restaurantes.to_parquet('./data/data_item_item.parquet', engine='pyarrow')